In [ ]:
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Conv2D, MaxPool2D, Flatten
from keras import optimizers
from keras import losses
from keras.utils.vis_utils import plot_model
from keras_preprocessing.image import ImageDataGenerator
import time, os, fnmatch, shutil

In [ ]:
cmmd_dir = "/media/craig/Larry/python/"

train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=20,
        shear_range=0.2,
        zoom_range=0.2,
        vertical_flip=True,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True)
train_generator = train_datagen.flow_from_directory(
        cmmd_dir+"cmmd_data/PNG/TRAIN/",
        target_size=(227, 227),
        color_mode="grayscale",
        batch_size=32,
        shuffle=True,
        class_mode='binary',
        seed=14)
validation_generator = test_datagen.flow_from_directory(
        cmmd_dir+"cmmd_data/PNG/VAL/",
        target_size=(227, 227),
        color_mode="grayscale",
        batch_size=32,
        shuffle=True,
        class_mode='binary',
        seed=14)

# Modified AlexNet implementation

In [ ]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=96, 
                        kernel_size=(11,11), 
                        strides=(4,4), 
                        activation='relu', 
                        input_shape=(227,227,1)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), 
                           strides=(2,2)),
    keras.layers.Conv2D(filters=256, 
                        kernel_size=(5,5), 
                        strides=(1,1), 
                        activation='relu', 
                        padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), 
                           strides=(2,2)),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), 
                        strides=(1,1), activation='relu', 
                        padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=384, kernel_size=(3,3), 
                        strides=(1,1), activation='relu', 
                        padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters=256, kernel_size=(3,3), 
                        strides=(1,1), activation='relu', 
                        padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPool2D(pool_size=(3,3), 
                           strides=(2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(4096, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])
# Uncomment for model summary
# model.summary()

In [ ]:
#Uncomment for PNG display of model architecture
#plot_model(model, to_file='alexnet_modified_model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
root_logdir = os.path.join(os.curdir, "my_logs")
def get_run_logdir():
    #TensorBosard run TITLE:
    run_id = ("AlexNet_Candidate_Model")
    return os.path.join(root_logdir,run_id)

run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

callbacks = [
    #comment next two lines to stop tensorboard if needed.
    keras.callbacks.ModelCheckpoint("/media/craig/Henry/ML_models/alexnet/save_at_{epoch}.h5"),
    keras.callbacks.TensorBoard(run_logdir)
]

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])
history = model.fit(train_generator, epochs=150, callbacks=callbacks, validation_data=validation_generator)

In [ ]:
model.save("alexnet_candidate_model.h5")